# OGGM glacier simulator

App to initialize a simple glacier and look how it develops under changing mass balance profiles.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from functools import partial
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline
from oggm.core.massbalance import LinearMassBalance
from oggm import cfg
cfg.initialize()

# There are several solvers in OGGM core. We use the default one for this experiment
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define some variables for model and plotting

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# default tools for plot
default_tools = ['save', 'wheel_zoom', 'box_zoom', 'reset']

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

# glacier range
max_glacier_range = np.linspace(glacier_bottom_height,
                                glacier_top_height, nx)

## Define values for slider:

In [ ]:
# ELA height
ELA_height_min = 0
ELA_height_max = 4000
ELA_height_default = 3000
ELA_height_step = 100

# mass balance gradient
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_step = 1

# years of calculation
years_min = 10
years_max = 1000
years_default = 150
years_step = 10

# linear bed rock slope
bed_rock_slope_values = ['steepest', 'steeper',
                         'steep', 'flat', 'flatter', 'flattest']
bed_rock_slope_default = 'flat'

# bed rock profile
bed_rock_profile_values = [
    'linear', 'getting flatter', 'getting steeper', 'cliff']
bed_rock_profile_default = 'linear'

# bed rock width
bed_rock_width_values = ['constant', 'getting narrower', 'getting wider',
                         'wide at the top and narrow at the bottom']
bed_rock_width_default = 'constant'

# Glen's creep parameter
glens_creep_parameter_values = ['small', 'default', 'large']
glens_creep_parameter_default = 'default'

# sliding parameter
sliding_parameter_values = ['no sliding', 'sliding']
sliding_parameter_default = 'no sliding'

## Define global model variables:

In [ ]:
bed_h = []
mb_model = []
model = []
run_surface_heights = []
glacier_outlines = []
glen_a = cfg.PARAMS['glen_a']
fs = 0

# define width, so glacier bed could be created when plot is created (after creation model is ready)
widths = np.zeros(nx) + 4.

run_loop_count = 0

## Define global curves:

In [ ]:
bed_rock_curve = []
glacier_height_curve = []
mb_curve = []
width_curve = []
info_text = []

## Define Panels

### Panel for beginner mode

#### beginner button function

In [ ]:
def beginner_button_click(arg=None):
    # set Variables not changeable in beginner mode to there default values
    mb_gradient.value = mb_gradient_default
    bed_rock_profile.value = bed_rock_profile_default
    glens_creep_parameter.value = glens_creep_parameter_default
    sliding_parameter.value = sliding_parameter_default

    # for consitency set values of the advanced mode
    ELA_height_advanced.value = ELA_height.value
    bed_rock_width_advanced.value = bed_rock_width.value
    bed_rock_slope_advanced.value = bed_rock_slope.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

    run_the_model(stage='beginner mode')

#### actual beginner Panel

In [ ]:
bed_rock_slope = pn.widgets.Select(name='slope of bedrock profile',
                                   options=bed_rock_slope_values,
                                   value=bed_rock_slope_default,
                                   sizing_mode='stretch_width')

bed_rock_width = pn.widgets.Select(name='width along glacier',
                                   options=bed_rock_width_values,
                                   value=bed_rock_width_default,
                                   sizing_mode='stretch_width')

ELA_height = pn.widgets.IntSlider(name='equilibrium line height',
                                  start=ELA_height_min,
                                  end=ELA_height_max,
                                  step=ELA_height_step,
                                  value=ELA_height_default,
                                  sizing_mode='stretch_width')

beginner_button = pn.widgets.Button(name='run the model',
                                    button_type='primary')
beginner_button.param.watch(beginner_button_click, 'clicks')

beginner_panel = pn.Column(pn.Row(bed_rock_slope, bed_rock_width, ELA_height),
                           beginner_button,
                           background='#fafafa',
                           sizing_mode='stretch_width')

### Panel for advanced mode

#### advanced button help functions

In [ ]:
def advance_model_click():
    # for consitency set values of the advanced mode
    ELA_height.value = ELA_height_advanced.value
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

#### advanced button functions

In [ ]:
def run_model_equilibrium_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance equilibrium')


def run_model_years_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance years')


def create_new_model_button_click(arg=None):
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    main_figure.event(button_name='new model',
                      clicks=create_new_model_button.clicks)

#### actual advanced Panel

In [ ]:
mb_gradient = pn.widgets.IntSlider(name='mass balance gradient',
                                   start=mb_gradient_min,
                                   end=mb_gradient_max,
                                   step=mb_gradient_step,
                                   value=mb_gradient_default,
                                   sizing_mode='stretch_width')

bed_rock_profile = pn.widgets.Select(name='bedrock profile',
                                     options=bed_rock_profile_values,
                                     value=bed_rock_profile_default,
                                     sizing_mode='stretch_width')

ELA_height_advanced = pn.widgets.IntSlider(name='equilibrium line height',
                                           start=ELA_height_min,
                                           end=ELA_height_max,
                                           step=ELA_height_step,
                                           value=ELA_height_default,
                                           sizing_mode='stretch_width')

bed_rock_slope_advanced = pn.widgets.Select(name='slope of linear bedrock profile',
                                            options=bed_rock_slope_values,
                                            value=bed_rock_slope_default,
                                            sizing_mode='stretch_width')

bed_rock_width_advanced = pn.widgets.Select(name='width along glacier',
                                            options=bed_rock_width_values,
                                            value=bed_rock_width_default,
                                            sizing_mode='stretch_width')

glens_creep_parameter = pn.widgets.Select(name="Glen's creep parameter",
                                          options=glens_creep_parameter_values,
                                          value=glens_creep_parameter_default,
                                          sizing_mode='stretch_width')

sliding_parameter = pn.widgets.RadioBoxGroup(name='Sliding parameter',
                                             options=sliding_parameter_values,
                                             value=sliding_parameter_default,
                                             inline=True,
                                             sizing_mode='stretch_width')

run_model_years = pn.widgets.IntSlider(name='years to advance model',
                                       start=years_min,
                                       end=years_max,
                                       step=years_step,
                                       value=years_default,
                                       sizing_mode='stretch_width')

run_model_equilibrium_button = pn.widgets.Button(name='run to equilibrium',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
run_model_equilibrium_button.param.watch(
    run_model_equilibrium_button_click, 'clicks')

run_model_years_button = pn.widgets.Button(name='advance model',
                                           button_type='primary',
                                           sizing_mode='stretch_width')
run_model_years_button.param.watch(run_model_years_button_click, 'clicks')

create_new_model_button = pn.widgets.Button(name='start new model',
                                            button_type='primary',
                                            sizing_mode='stretch_width')
create_new_model_button.param.watch(create_new_model_button_click, 'clicks')

advanced_panel = pn.Tabs(('run the model',
                          pn.Row(pn.Column(ELA_height_advanced,
                                           mb_gradient,
                                           sizing_mode='stretch_width'),
                                 pn.Column(glens_creep_parameter,
                                           sliding_parameter,
                                           sizing_mode='stretch_width'),
                                 pn.Column(run_model_years,
                                           sizing_mode='stretch_width'),
                                 pn.Column(run_model_years_button,
                                           run_model_equilibrium_button,
                                           sizing_mode='stretch_width'),
                                 sizing_mode='stretch_width')),
                         ('create new model',
                          pn.Column(pn.Row(bed_rock_profile,
                                           bed_rock_slope_advanced,
                                           bed_rock_width_advanced),
                                    create_new_model_button,
                                    sizing_mode='stretch_width')),
                         background='#fafafa')

### Panel for help

#### help functions for buttons

In [ ]:
def beginner_introduction_button_click(arg=None):
    pass


def advanced_introduction_button_click(arg=None):
    pass


def show_subfigure_explenation_button_click(arg=None):
    pass


def show_parameter_explenation_button_click(arg=None):
    pass

#### actual help Panel

In [ ]:
help_intro_text = pn.pane.Markdown('Here you can find help for the different modes and explenations what the figures ' +
                                   'are showing and what each changeable parameter stands for. Click the button you ' +
                                   'would like to know more ;)',
                                   sizing_mode='stretch_both')

beginner_introduction_button = pn.widgets.Button(disabled=True,
                                                 name='beginner mode introduction',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
beginner_introduction_button.param.watch(
    beginner_introduction_button_click, 'clicks')

advanced_introduction_button = pn.widgets.Button(disabled=True,
                                                 name='advanced mode introduction',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
advanced_introduction_button.param.watch(
    advanced_introduction_button_click, 'clicks')

show_subfigure_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of each subfigure',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_subfigure_explenation_button.param.watch(
    show_subfigure_explenation_button_click, 'clicks')

show_parameter_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of parameters',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_parameter_explenation_button.param.watch(
    show_parameter_explenation_button_click, 'clicks')

help_panel = pn.Row(help_intro_text,
                    pn.Column(beginner_introduction_button,
                              advanced_introduction_button),
                    pn.Column(show_subfigure_explenation_button,
                              show_parameter_explenation_button),
                    background='#fafafa',
                    sizing_mode='stretch_width')

### add all panels together as tabs

In [ ]:
tab_menu = pn.Tabs(('beginner mode', beginner_panel),
                   ('advanced mode', advanced_panel),
                   ('find help here (under construction)', help_panel))

## Define functions to set numeric values of parameters

In [ ]:
def set_glen_a():
    global glen_a

    if glens_creep_parameter.value == 'default':
        glen_a = cfg.PARAMS['glen_a']
    elif glens_creep_parameter.value == 'small':
        glen_a = cfg.PARAMS['glen_a'] / 10
    elif glens_creep_parameter.value == 'large':
        glen_a = cfg.PARAMS['glen_a'] * 10

In [ ]:
def set_fs():
    global fs

    if sliding_parameter.value == 'no sliding':
        fs = 0
    elif sliding_parameter.value == 'sliding':
        fs = 5.7e-20

## Define function for creating curves

In [ ]:
def set_bed_rock_curve():
    global bed_h
    global bed_rock_curve

    # look which bed rock profile was chosen
    if bed_rock_profile.value == 'linear':
        # look which slope was choosen and set the heigth at each point along the glacier
        if bed_rock_slope.value == 'steepest':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4)),
                              np.zeros(nx - np.int(np.round(nx / 4))))
        elif bed_rock_slope.value == 'steeper':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 2),
                              np.zeros(nx - np.int(np.round(nx / 4) * 2)))
        elif bed_rock_slope.value == 'steep':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 3),
                              np.zeros(nx - np.int(np.round(nx / 4) * 3)))
        elif bed_rock_slope.value == 'flat':
            bed_h = np.linspace(glacier_top_height, 0, nx)
        elif bed_rock_slope.value == 'flatter':
            bed_h = np.linspace(glacier_top_height, 1500, nx)
        elif bed_rock_slope.value == 'flattest':
            bed_h = np.linspace(glacier_top_height, 3000, nx)

    elif bed_rock_profile.value == 'getting flatter':
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'getting steeper':
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) + 1.3) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'cliff':
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400

        bed_h = np.concatenate((np.linspace(glacier_top_height, cliff_top, nx/2),
                                np.linspace(cliff_bottom, glacier_bottom_height, nx/2)))

    # create curve of bed rock
    bed_rock_curve = hv.Area((distance_along_glacier, bed_h),
                             'distance along glacier (km)', 'altitude (m)',
                             ).opts(default_tools=default_tools,
                                    color='lightgray',
                                    line_alpha=0)

In [ ]:
def set_glacier_height_curve():
    global model
    global run_surface_heights
    global glacier_height_curve

    ELA = int(ELA_height.value)

    glacier_height_curve = (hv.Area((distance_along_glacier, run_surface_heights[0]),
                                    'distance along glacier (km)', 'altitude (m)',
                                    label='old').opts(default_tools=default_tools,
                                                      line_dash='dashed',
                                                      line_color='darkgray',
                                                      color='white',
                                                      alpha=0.6,
                                                      line_alpha=1,
                                                      line_width=3) *
                            hv.Area((distance_along_glacier, run_surface_heights[1]),
                                    'distance along glacier (km)', 'altitude (m)',
                                    label='new').opts(default_tools=default_tools,
                                                      color='white',
                                                      line_color='blue',
                                                      alpha=0.6,
                                                      line_alpha=1,
                                                      line_width=3) *
                            hv.Curve((distance_along_glacier, np.repeat(ELA, np.size(distance_along_glacier))),
                                     'distance along glacier (km)',
                                     'altitude (m)'
                                     ).opts(default_tools=default_tools,
                                            line_dash='dashed',
                                            line_width=3,
                                            color='black')
                            )

In [ ]:
def set_mb_curve():
    global mb_model
    global mb_curve

    ELA = int(ELA_height.value)
    gradient = int(mb_gradient.value)

    mb_model = LinearMassBalance(ELA, grad=gradient)

    # define the range of the mass balance curve wich should be shown
    y_range_of_mb_curve = np.array([-20, 20])
    x_range_of_mb_curve = np.array([0, 4500])

    # get the annual mass balance for the given range of the x axis
    annual_mb = mb_model.get_annual_mb(x_range_of_mb_curve) * cfg.SEC_IN_YEAR

    mb_curve = (hv.Area((y_range_of_mb_curve, np.repeat(np.max(x_range_of_mb_curve), 2)),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass gain'
                        ).opts(default_tools=default_tools,
                               color='lightgreen',
                               line_alpha=0) *
                hv.Area((y_range_of_mb_curve, np.repeat(ELA, 2)),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass loss'
                        ).opts(default_tools=default_tools,
                               color='lightcoral',
                               line_alpha=0) *
                hv.Curve((annual_mb, x_range_of_mb_curve),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='mass balance'
                         ).opts(default_tools=default_tools,
                                color='black') *
                hv.Curve((y_range_of_mb_curve, np.repeat(ELA, 2)),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='ELA'
                         ).opts(default_tools=default_tools,
                                line_dash='dashed',
                                line_width=3,
                                color='black')
                )

In [ ]:
def set_width_curve():
    global widths
    global width_curve
    global model
    global glacier_outlines

    # look which glacier width was selected
    if bed_rock_width.value == 'constant':
        widths = np.zeros(nx) + 4.

    elif bed_rock_width.value == 'getting wider':
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'getting narrower':
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'wide at the top and narrow at the bottom':
        widths = np.append(np.ones(np.int(np.round(nx/2))) * 4,
                           np.ones(np.int(nx - np.round(nx/2))) * 2)

    # define lower and upper y limit which is shown
    y_min = -5  # in m
    y_max = 3

    # actual glacier width plot
    width_curve = (hv.Area((distance_along_glacier, y_min),
                           'distance along glacier (km)',
                           'width (m)',
                           # label='boarder of glacier rock bed'
                           ).opts(default_tools=default_tools,
                                  color='darkgray',
                                  line_alpha=0) *
                   hv.Area((distance_along_glacier, y_max),
                           'distance along glacier (km)',
                           'width (m)',
                           # label='boarder of glacier rock bed'
                           ).opts(default_tools=default_tools,
                                  color='darkgray',
                                  line_alpha=0) *
                   hv.Area((distance_along_glacier, -widths/2),
                           'distance along glacier (km)',
                           'width (m)',
                           # label='glacier rock bed'
                           ).opts(default_tools=default_tools,
                                  color='lightgray',
                                  line_alpha=0) *
                   hv.Area((distance_along_glacier, widths/2),
                           'distance along glacier (km)',
                           'width (m)',
                           # label='glacier rock bed'
                           ).opts(default_tools=default_tools,
                                  color='lightgray',
                                  line_alpha=0) *
                   hv.Area((distance_along_glacier, -glacier_outlines[0]/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='old'
                           ).opts(default_tools=default_tools,
                                  line_dash='dashed',
                                  color='white',
                                  line_color='darkgray',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Area((distance_along_glacier, glacier_outlines[0]/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='old'
                           ).opts(default_tools=default_tools,
                                  line_dash='dashed',
                                  color='white',
                                  line_color='darkgray',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Area((distance_along_glacier, -glacier_outlines[1]/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='new'
                           ).opts(default_tools=default_tools,
                                  color='white',
                                  line_color='blue',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Area((distance_along_glacier, glacier_outlines[1]/2),
                           'distance along glacier (km)',
                           'width (m)',
                           label='new'
                           ).opts(default_tools=default_tools,
                                  color='white',
                                  line_color='blue',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                            'distance along glacier (km)',
                            'width (m)',
                            label='center Flowline'
                            ).opts(default_tools=default_tools,
                                   color='black')
                   ).opts(ylim=(y_min, y_max))

In [ ]:
def set_info_text(stage='new model'):
    global info_text
    global model

    if stage == 'new model':
        text = 'New Model geometry\ninitialised'
    elif stage == 'running model':
        if model.volume_km3 > 1:
            text = 'Model is running\n{:.0f} years\n with a Volume of\n{:.2f} km³'.format(
                model.yr, model.volume_km3)
        else:
            text = 'Model is running\n{:.0f} years\n with a Volume of\n{:.0f} m³'.format(
                model.yr, model.volume_m3)
    elif stage == 'equilibrium':
        if model.volume_km3 > 1:
            text = 'Model reached equilibrium\n{:.0f} years\n with a Volume of\n{:.2f} km³'.format(
                model.yr, model.volume_km3)
        else:
            text = 'Model reached equilibrium\n{:.0f} years\n with a Volume of\n{:.0f} m³'.format(
                model.yr, model.volume_m3)
    elif stage == 'stopped model':
        if model.volume_km3 > 1:
            text = 'Model finished\n{:.0f} years\n with a Volume of\n{:.2f} km³'.format(
                model.yr, model.volume_km3)
        else:
            text = 'Model finished\n{:.0f} years\n with a Volume of\n{:.0f} m³'.format(
                model.yr, model.volume_m3)

    info_text = hv.Text(0.5, 0.5, text).opts(default_tools=['save', 'reset'],
                                             show_frame=False,
                                             text_baseline='middle',
                                             text_align='center',
                                             xaxis=None,
                                             yaxis=None)

## Define functions for intialize flowline and run model

In [ ]:
def init_flowline_model():
    global model
    global bed_h
    global run_surface_heights
    global widths
    global glacier_outlines

    # set info text
    set_info_text(stage='new model')

    # initialize the bed rock
    set_bed_rock_curve()

    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with rectangular shape of glacier
    model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                            widths=widths, map_dx=map_dx)

    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0., glen_a=glen_a,
                          fs=fs, check_for_boundaries=False)

    # set surface height for plotting, at beginn old and new one are the same
    run_surface_heights = np.repeat(
        np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)

    # set glacier outlines for plottin, at beginn old and new one are the same
    glacier_outlines = np.repeat(np.array(
        np.where(model.fls[-1].thick > 0, widths, np.zeros(nx)), ndmin=2), 2, axis=0)

In [ ]:
def run_the_model(stage='beginner mode'):
    global model
    global run_surface_heights
    global run_loop_count
    global glacier_outlines

    # set new mass balance curve
    set_mb_curve()

    if stage == 'beginner mode':
        # initialize a new model
        init_flowline_model()

        # years after which the current state of the glacier should be shown
        dyears = 10

    elif stage == 'advance equilibrium' or stage == 'advance years':
        # initialize model with old extend and new mass balance model
        model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.,
                              glen_a=glen_a, fs=fs, check_for_boundaries=False)

        # determine glacier state at starting point
        run_surface_heights = np.repeat(
            np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)
        glacier_outlines = np.repeat(np.array(
            np.where(model.fls[-1].thick > 0, widths, np.zeros(nx)), ndmin=2), 2, axis=0)

        # TODO: create slider to change dyears in advanced mode
        dyears = 10

    if stage == 'beginner mode':
        set_glacier_height_curve()
        set_width_curve()

    if stage == 'beginner mode' or stage == 'advance equilibrium':
        # same function as from OGGM FlowlineModel.run_until_equilibrium
        ite = 0
        was_close_zero = 0
        t_rate = 1
        while (t_rate > 0.001) and (ite <= 200) and (was_close_zero < 5):
            ite += 1
            v_bef = model.volume_m3
            model.run_until(model.yr + dyears)
            v_af = model.volume_m3
            if np.isclose(v_bef, 0., atol=1):
                t_rate = 1
                was_close_zero += 1
            else:
                t_rate = np.abs(v_af - v_bef) / v_bef

            # set text with current year and volume
            set_info_text(stage='running model')

            # save new surface/outline and show it on plot
            run_surface_heights[1] = np.array(model.fls[-1].surface_h, ndmin=2)
            glacier_outlines[1] = np.array(
                np.where(model.fls[-1].thick > 0, widths, np.zeros(nx)), ndmin=2)
            set_glacier_height_curve()
            set_width_curve()
            main_figure.event(button_name='run_button',
                              clicks=run_loop_count)
            run_loop_count += 1

        # TODO: Insert message when equilibrium could not be found
        # if ite > max_ite:

        # set text with equilibrium year
        set_info_text(stage='equilibrium')
        main_figure.event(button_name='run_button',
                          clicks=run_loop_count)

    elif stage == 'advance years':
        while (model.yr < run_model_years.value):
            model.run_until(model.yr + dyears)

            # set text with current year and volume
            set_info_text(stage='running model')

            # save new surface/outline and show it on plot
            run_surface_heights[1] = np.array(model.fls[-1].surface_h, ndmin=2)
            glacier_outlines[1] = np.array(
                np.where(model.fls[-1].thick > 0, widths, np.zeros(nx)), ndmin=2)
            set_glacier_height_curve()
            set_width_curve()
            main_figure.event(button_name='run_button',
                              clicks=run_loop_count)
            run_loop_count += 1

        # TODO: Insert message when equilibrium could not be found
        # if ite > max_ite:

        # set text with equilibrium year
        set_info_text(stage='stopped model')
        main_figure.event(button_name='run_button',
                          clicks=run_loop_count)

## Main function for dynamic map

In [ ]:
def change_plot(button_name='reset', clicks=0):
    # use global curves
    global bed_rock_curve
    global glacier_height_curve
    global mb_curve
    global width_curve

    if button_name == 'reset':
        # TODO: set all widgets to default values
        # bed_rock_profile.value = 'linear'

        # initialisation of figure
        set_mb_curve()
        init_flowline_model()
        set_glacier_height_curve()
        set_width_curve()

    elif button_name == 'new model':
        set_mb_curve()
        init_flowline_model()
        set_glacier_height_curve()
        set_width_curve()

    return ((glacier_height_curve * bed_rock_curve).opts(legend_position='bottom_left',
                                                         legend_cols=2,
                                                         xaxis='top',
                                                         bgcolor='lightblue',
                                                         frame_width=580,
                                                         frame_height=200) +
            (mb_curve).opts(legend_position='right',
                            xaxis='top',
                            yaxis='right',
                            frame_width=200,
                            frame_height=200) +
            (width_curve).opts(legend_cols=3,
                               legend_position='bottom_left',
                               frame_height=150,
                               frame_width=580,
                               yformatter='%8.0f') +
            (info_text).opts(frame_height=150,
                             frame_width=300)
            ).cols(2)

## streamer and dynamic map

In [ ]:
main_stream = Stream.define('main_stream', button_name='reset', clicks=0)

main_figure = hv.DynamicMap(change_plot, streams=[main_stream()])

## App

In [ ]:
pn.Column(tab_menu, main_figure, sizing_mode='stretch_both').servable()

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.